In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
def read_vocab(filename):
    
    dict_ = {}
    with open(filename, 'r', encoding='utf-8-sig') as datafile:
        for line in datafile:
            occurence = line.strip().split(' ')[0]
            word = line.strip().split(' ')[1]
            dict_[word] = int(occurence)
    return dict_

In [3]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/twitter_datasets_epfl/full/')

In [4]:
#  vocab pos
os.system('build_vocab_pos_full.sh')
os.system('cut_vocab_pos_full.sh')

#  vocab neg
os.system('build_vocab_neg_full.sh')
os.system('cut_vocab_neg_full.sh')

-1073741819

In [32]:
# read the vocabs
vocab_neg = read_vocab('vocab_neg_full.txt')
vocab_pos = read_vocab('vocab_pos_full.txt')

In [33]:
import operator

def extract_most_freq(vocab_dict, max_words=10):
    
    most_freq_words = []
    vocab_list_sorted = sorted(vocab_dict.copy().items(), key=operator.itemgetter(1), reverse = True)

    for i in range(max_words):
        most_freq_words.append((vocab_list_sorted[i][0], vocab_list_sorted[i][1]))
    
    return most_freq_words

In [34]:
most_neg = extract_most_freq(vocab_neg, max_words=10)
print("\n".join("{}\t{}".format(x[0], x[1]) for x in most_neg))

.	751865
i	625066
<user>	557268
(	461668
<number>	420633
<repeat>	414073
the	384681
<url>	359419
,	357807
!	356189


In [35]:
most_pos = extract_most_freq(vocab_pos, max_words=10)
print("\n".join("{}\t{}".format(x[0], x[1]) for x in most_pos))

﻿<user>	951004
!	585557
i	543778
.	499484
you	386420
to	317352
,	305792
the	297102
a	239967
is	239662


In [43]:
def word_ratio_neg_pos(vocab_dict_neg, vocab_dict_pos):
    
    word_ratio = {}

    # neg
    for word_neg, occu_neg in vocab_dict_neg.items():  
        
        # if word in both neg and pos vocab
        if word_neg in vocab_dict_pos:
            
            occu_pos = vocab_dict_pos.get(word_neg)
            word_ratio[word_neg] = [occu_neg/occu_pos, occu_neg, occu_pos]
            
    return word_ratio

In [79]:
freq_min = 100
vocab_neg_thr = {k: v for k, v in vocab_neg.copy().items() if v > freq_min}
vocab_pos_thr = {k: v for k, v in vocab_pos.copy().items() if v > freq_min}

In [80]:
word_ratio = word_ratio_neg_pos(vocab_neg_thr, vocab_pos_thr)

In [109]:
print(word_ratio.get('happy'))
print(word_ratio.get('not'))
print(word_ratio.get('honestly'))

[0.2264985696227796, 6809, 30062]
[1.5165902801165565, 207145, 136586]
[1.0843570843570844, 1324, 1221]


In [104]:
# build exclusion vocab with stop words

ratio_margin = 0.2
freq_min = 100
stop_vocab = {}
for word, values in word_ratio.items():
    
    ratio = values[0]
    occu_neg = values[1]
    occu_pos = values[2]
    
    # remove non discriminant word
    if ratio > 1 - ratio_margin and ratio < 1 + ratio_margin:
        stop_vocab[word] = values
        
    # remove unfrequent word
    if occu_neg < freq_min and occu_pos < freq_min:
        stop_vocab[word] = values
    
print("Number of stop words: ", len(stop_vocab))

Number of stop words:  1207


In [105]:
print("\n".join("{}\t{}".format(k,v) for k,v in stop_vocab.items()))

'	[1.040695000985351, 31685, 30446]
&	[0.8287230323098123, 40295, 48623]
*	[0.9180648876943751, 23911, 26045]
,	[1.1700992831728756, 357807, 305792]
;	[0.8940941646119526, 7615, 8517]
\	[0.9848208864602307, 1622, 1647]
_	[0.9908675799086758, 5208, 5256]
__	[0.9632107023411371, 864, 897]
`	[0.8138297872340425, 918, 1128]
{	[1.0107142857142857, 566, 560]
}	[0.868824531516184, 510, 587]
~	[0.9071597167584579, 4612, 5084]
<elong>	[0.8696475937855248, 87210, 100282]
<number>d	[1.0753828032979977, 2739, 2547]
<number>day	[0.9006342494714588, 426, 473]
<number>days	[0.851063829787234, 280, 329]
<number>ish	[0.8195488721804511, 109, 133]
<number>th	[0.8775769556622423, 6215, 7082]
<number>years	[0.983739837398374, 121, 123]
<number>yrs	[1.0833333333333333, 130, 120]
=	[0.9622151716181137, 3336, 3467]
->	[0.9617486338797814, 352, 366]
a	[0.9637950218154996, 231279, 239967]
aaaw	[0.9, 135, 150]
aah	[1.0, 191, 191]
aaron	[1.1644736842105263, 177, 152]
abby	[0.9107142857142857, 102, 112]
above	[0.

In [106]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/stop_words/')

In [107]:
# write stop word in a text
outputfilename = "stop_word_" + "freq_min_" + str(freq_min) + "_ratio_marg_" + str(ratio_margin) + ".txt"
with open(outputfilename, 'w', encoding='utf-8-sig') as outfile:
    for word, value in stop_vocab.items():
        outfile.write(word)   
        outfile.write("\n")      
outfile.close()